# VOTADECK
## Priest Control
Fast analysis of the metagame for Priest Control archetype

In [78]:
import pandas as pd
import numpy as np

Load the file with the frequency of deck archetypes in the game into the decks dataframe

Load the file with the frequency of the heros in the game into the heros dataframe

*this data was hand gathered from vS-DataReaper

In [126]:
decks = pd.read_csv("datasets\deck_presence.csv")
heros = pd.read_csv("datasets\hero_presence.csv")

Explore the head of the decks dataframe

In [127]:
decks.head()

,hero,deck,percent
0,Shaman,Midrange,20.75
1,Shaman,Control,1.12
2,Shaman,Aggro,0.93
3,Shaman,Other,0.80
4,Shaman,Evolve,0.48


Sort the decks by its frequency in the meta

In [88]:
decks = decks.sort_values('percent', ascending=False)
decks.head()

,hero,deck,percent
0,Shaman,Midrange,20.75
14,Mage,Tempo,7.42
19,Druid,Spell,7.04
5,Hunter,Secret,6.83
23,Warlock,Zoo,5.73


## Only the top 80% more frequent decks
Lets eliminate some decks (most of them) and keep only those that comprehend the 80% more frequent decks of the field

In [89]:
n = [n for n in decks['percent']]
val = 80
for count, tot in zip(range(len(decks)), decks['percent']):
    val -= tot
    if val <= 0:
        print(count)
        break
print(decks.iloc[count][['hero','deck','percent']])

15
hero       Druid
deck       Beast
percent     1.97
Name: 20, dtype: object


Save the top decks in a separated dataframe

In [90]:
topdecks = decks.iloc[0:count+1]

Loads the matchup data from file to dataframe matchup and explores it.

## Matchup against the top decks
Now we will categorize the games, defining how likely it is for a game against one of the top decks is to be won or lost, based on the matchups file.

In [91]:
matchup = pd.read_csv("datasets/priest_control_matchup.csv")
matchup.head()

,hero,deck,win_chance
0,Druid,Beast,0.3600
1,Druid,CThun,0.4959
2,Druid,Spell,0.4118
3,Hunter,Face,0.3488
4,Hunter,Midrange,0.4042


Merge the topdecks and matchup dataframes into a new one rl_decks, which will hold both chance to win against an archetype and the percentage a given deck appears in the field

In [113]:
rl_decks = pd.merge(topdecks, matchup, on=["hero","deck"], how="inner")
rl_decks

,hero,deck,percent,win_chance
0,Shaman,Midrange,20.75,0.3884
1,Mage,Tempo,7.42,0.4056
2,Druid,Spell,7.04,0.4118
3,Hunter,Secret,6.83,0.4234
4,Warlock,Zoo,5.73,0.5039
5,Rogue,Miracle,5.16,0.3602
6,Hunter,Face,4.34,0.3488
7,Warrior,Control,4.22,0.5851
8,Warrior,CThun,2.90,0.4766
9,Warrior,Pirate,2.87,0.4417


Correction of the frequency from ~0.78 to 1.0, the objective in this correction is to normalize the deck frequency distribution to this new study set.

In [128]:
rl_decks['percent'] = rl_decks['percent']/100
rl_decks['percent'] = rl_decks['percent'] * (1 / sum(rl_decks['percent']))

Separating the archetypes in two groups, those which the victory chance is equal or bigger than 0.43, ans those that are less.

In [115]:
w = rl_decks[rl_decks['win_chance'] >= 0.43]
l = rl_decks[rl_decks['win_chance'] < 0.43]

Weighted average of the chance to win of Priest Control agains the study group.

In [117]:
sum(rl_decks['percent'] * rl_decks['win_chance'])

0.41769829906067529

### Weighted average of the winning-rate against 'W' group
Group W, archetypes which the win-rate against them is higher or equal to 0.43

In [122]:
w['normalized_percent'] = w['percent'] * (1 / sum(w['percent']))
sum(w['normalized_percent'] * w['win_chance'])

c:\python35\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


0.50056213808463246

In [129]:
w #archetypes

,hero,deck,percent,win_chance,normalized_percent
4,Warlock,Zoo,0.072734,0.5039,0.319042
7,Warrior,Control,0.053567,0.5851,0.234967
8,Warrior,CThun,0.036811,0.4766,0.161470
9,Warrior,Pirate,0.036431,0.4417,0.159800
12,Warlock,Reno,0.028434,0.4392,0.124722


### Weighted average of the winning-rate against 'L' group
Group W, archetypes which the win-rate against them is lower than 0.43

In [123]:
l['normalized_percent'] = l['percent'] * (1 / sum(l['percent']))
sum(l['normalized_percent'] * l['win_chance'])

c:\python35\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


0.3932288063137126

In [125]:
l #archetypes

,hero,deck,percent,win_chance,normalized_percent
0,Shaman,Midrange,0.263392,0.3884,0.341171
1,Mage,Tempo,0.094186,0.4056,0.121999
2,Druid,Spell,0.089363,0.4118,0.115751
3,Hunter,Secret,0.086697,0.4234,0.112299
5,Rogue,Miracle,0.065499,0.3602,0.084841
6,Hunter,Face,0.055090,0.3488,0.071358
10,Hunter,Midrange,0.034400,0.4042,0.044558
11,Warrior,Dragon,0.032242,0.3963,0.041763
13,Paladin,Anyfin,0.026149,0.4237,0.033870
14,Druid,Beast,0.025006,0.3600,0.032391
